## Hartree-Fock using PySCF

In the lectures we have covered the Hartree-Fock method of finding an approximation to the ground-state energy of the electrons in a molecule.  While it is theoretically possible to solve the relevant equations by hand, this is exceptionally long and tedious.  Instead, we typically use one of the many quantum chemistry packages which have been developed for the purpose of performing this procedure on a computer.  In this problem, we will use the pyscf package for this purpose.  First, we install the package (probably best to do this in a fresh environment):

In [1]:
import sys
!{sys.executable} -m pip install pyscf
import pyscf

  Using cached pyscf-2.1.1.tar.gz (8.5 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for pyscf: filename=pyscf-2.1.1-cp39-cp39-macosx_11_0_arm64.whl size=53190929 sha256=dd23ef0c9ed6e966c3617b59e87783d601a11ed42d0bbd34be6dadb593d27e47
  Stored in directory: /Users/nathanfitzpatric/Library/Caches/pip/wheels/71/48/52/37524c67675ed82f1da461bd14c9285c0e8b9fdc310734ed3b
Successfully built pyscf

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


We first need to tell PySCF what molecule we're interested in.  See if you can figure out what the two lines below mean:

In [11]:
geometry = 'H 0. 0. 0.; H 0. 0. 0.714'
basis = 'STO-3G'
spin = 

Now we'll create the PySCF object which represents the molecule (don't ask us why the parameter is called "atom" instead of "molecule).

In [20]:
mol = pyscf.gto.M(atom = geometry, basis = basis)

This is all we'll need to be able to run a simple Hartree-Fock calculation.  Of course, there are many options to tweak how the process will run, but we'll ignore them for now.  To run Hartree-Fock, we create an scf.RHF object and then use the kernel method to get the energy (in Hartrees):

In [21]:
hartree_fock = pyscf.scf.RHF(mol)
print(type(hartree_fock))
hf_energy = hartree_fock.kernel()
print(hf_energy)

<class 'pyscf.scf.hf.RHF'>
converged SCF energy = -1.11750269105473
-1.1175026910547303


In [23]:
from pyscf.tools import cubegen
from pyscf import gto, scf

mf = scf.RHF(mol).run()

cubegen.density(mol, 'h2o_den.cube', mf.make_rdm1())

# electron density slice
cubegen.density(mol, 'h2o_den_slice.cube', mf.make_rdm1(), nx=1, ny=1, nz=80)

# molecular electrostatic potential
cubegen.mep(mol, 'h2o_pot.cube', mf.make_rdm1())

# molecular electrostatic potential slice
cubegen.mep(mol, 'h2o_pot_slice.cube', mf.make_rdm1(), nx=1, ny=1, nz=80)

# 1st MO
cubegen.orbital(mol, 'h2o_mo1.cube', mf.mo_coeff[:,0]).

# 1st MO orbital slice
cubegen.orbital(mol, 'h2o_mo1_slice.cube', mf.mo_coeff[:,0], nx=1, ny=1, nz=80)

converged SCF energy = -1.11750269105473


array([[[0.00056676, 0.00062612, 0.00068985, 0.00075804, 0.00083075,
         0.00090802, 0.00098986, 0.00107623, 0.00116707, 0.00126225,
         0.00136162, 0.00146499, 0.00157211, 0.00168269, 0.0017964 ,
         0.00191284, 0.0020316 , 0.0021522 , 0.00227413, 0.00239683,
         0.00251973, 0.0026422 , 0.00276361, 0.00288329, 0.00300057,
         0.00311476, 0.00322518, 0.00333114, 0.00343196, 0.00352701,
         0.00361565, 0.00369729, 0.00377136, 0.00383737, 0.00389484,
         0.00394337, 0.00398263, 0.00401232, 0.00403223, 0.00404222,
         0.00404222, 0.00403223, 0.00401232, 0.00398263, 0.00394337,
         0.00389484, 0.00383737, 0.00377136, 0.00369729, 0.00361565,
         0.00352701, 0.00343196, 0.00333114, 0.00322518, 0.00311476,
         0.00300057, 0.00288329, 0.00276361, 0.0026422 , 0.00251973,
         0.00239683, 0.00227413, 0.0021522 , 0.0020316 , 0.00191284,
         0.0017964 , 0.00168269, 0.00157211, 0.00146499, 0.00136162,
         0.00126225, 0.00116707, 0

## Full configuration interaction with PySCF

As we discussed in the lectures, Hartree-Fock is only one of many computational methods in chemistry to estimate the ground state energy of the electrons in a molecule.  There is typically a trade-off between the accuracy of the method, and the computational difficulty (though perhaps not the conceptual difficulty!).  

In later lectures, we'll cover the Full Configuration Interaction method.  This is at the extreme end of the difficulty vs. accuracy method -- it is in fact an exact diagonalization of the electronic Hamiltonian.  As such, it is numerically exact, but exponentially hard (if it were not exponentially hard, we wouldn't need quantum computers and we'd be out of a job).  Due to the computational difficulty, it is not used much in earnest, but for small systems can be used as a benchmark for testing how accurate a computational method is.

As we'll come on to in later lectures, FCI requires a *reference state* to start from.  This is an approximate solution to the ground state.  In this instance, we can use our Hartree-Fock solution:

In [8]:
from pyscf import fci
full_configuration_interaction = pyscf.fci.FCI(hartree_fock)

Now we can just tell PySCF to run the FCI algorithm:

In [9]:
full_configuration_interaction.run()
fci_energy = full_configuration_interaction.kernel()[0]
print(fci_energy)

-1.1369181758408116


We know that the FCI energy is exact.  So if we compare our Hartree-Fock energy against our FCI energy, we obtain the error caused by the approximations made in the Hartree-Fock procedure.  

In [10]:
print(fci_energy-hf_energy)

-0.01941548478608124


Note how the FCI energy is lower than the HF energy, in agreement with the variational principle.  Also note that the error is actually quite small - it is common in chemistry for the error incurred by approximation to be maybe ~1% of the full energy, although this is very system dependent.  The danger is that these small errors can easily become large errors when propagated through a successor calculation.